In [1]:
import pandas as pd
import requests
from collections import Counter
import datetime as dt

In [2]:
#token given by URA
ura_access_key = 'd8722f05-25ff-44f6-bb6f-5d728aa4c9b1'

#getting the token for the day
r = requests.get("https://www.ura.gov.sg/uraDataService/insertNewToken.action", headers={"AccessKey":ura_access_key})
token = r.json()['Result']

In [3]:
#accessing the data in 4 batches as required by URA API
for i in range(1,5):
    #requesting data by passing in access key and token, in 4 batches as stated in URA API website
    req = requests.get('https://www.ura.gov.sg/uraDataService/invokeUraDS?service=PMI_Resi_Transaction&batch=' + str(i),
                      headers={"AccessKey":ura_access_key, "Token":token})
    
    df_temp = pd.json_normalize(req.json()['Result'], 'transaction', ['street','x','y','project'],errors='ignore')

    #adding each batch to DataFrame
    if i==1:
        df = df_temp
    else:
        df = df.append(df_temp, ignore_index=True) #ignore index so that append can be done
    
    #printing the progress for monitoring
    print('Batch ' + str(i) + ' completed')

Batch 1 completed
Batch 2 completed
Batch 3 completed
Batch 4 completed


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65022 entries, 0 to 65021
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   area          65022 non-null  object
 1   floorRange    65022 non-null  object
 2   noOfUnits     65022 non-null  object
 3   contractDate  65022 non-null  object
 4   typeOfSale    65022 non-null  object
 5   price         65022 non-null  object
 6   propertyType  65022 non-null  object
 7   district      65022 non-null  object
 8   typeOfArea    65022 non-null  object
 9   tenure        65022 non-null  object
 10  nettPrice     245 non-null    object
 11  street        65022 non-null  object
 12  x             43740 non-null  object
 13  y             43740 non-null  object
 14  project       65022 non-null  object
dtypes: object(15)
memory usage: 7.4+ MB


In [5]:
#saving to local drive
df.to_csv("d:/downloads/ura_caveats_2015-10 to 2020-09 raw data")

df1 = df.copy()


In [6]:
df1.area = df1.area.astype('float') * 10.76
print(df1.info())

df1 = df1.astype({'area':'int32','noOfUnits':'int32','floorRange':'category','typeOfSale':'category',
               'price':'int','propertyType':'category','district':'category','typeOfArea':'category',
               'tenure':'str','x':'float','y':'float'})

df1['Unit Price psf'] = (df1['price']/df1['area']).astype('int')

df1.contractDate = pd.to_datetime(df1.contractDate, format='%m%y').dt.to_period('M')

print(df1.info())
display(df1.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65022 entries, 0 to 65021
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          65022 non-null  float64
 1   floorRange    65022 non-null  object 
 2   noOfUnits     65022 non-null  object 
 3   contractDate  65022 non-null  object 
 4   typeOfSale    65022 non-null  object 
 5   price         65022 non-null  object 
 6   propertyType  65022 non-null  object 
 7   district      65022 non-null  object 
 8   typeOfArea    65022 non-null  object 
 9   tenure        65022 non-null  object 
 10  nettPrice     245 non-null    object 
 11  street        65022 non-null  object 
 12  x             43740 non-null  object 
 13  y             43740 non-null  object 
 14  project       65022 non-null  object 
dtypes: float64(1), object(14)
memory usage: 7.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65022 entries, 0 to 65021
Data columns (total 16 col

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,street,x,y,project,Unit Price psf
0,3378,-,1,2018-06,3,4750000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,22726.561999,29491.915576,LANDED HOUSING DEVELOPMENT,1406
1,3314,-,1,2018-09,3,5000000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,22726.561999,29491.915576,LANDED HOUSING DEVELOPMENT,1508
2,5641,-,1,2020-02,3,5500000,Semi-detached,05,Land,Freehold,NaN,ZEHNDER ROAD,22726.561999,29491.915576,LANDED HOUSING DEVELOPMENT,975
3,1714,-,1,2020-03,3,2630000,Terrace,05,Land,Freehold,NaN,NEO PEE TECK LANE,20862.625636,30610.450678,LANDED HOUSING DEVELOPMENT,1534
4,3744,06-10,1,2018-09,3,4400000,Condominium,04,Strata,99 yrs lease commencing from 2007,NaN,COVE DRIVE,28384.068816,25005.804519,TURQUOISE,1175


In [7]:
display(df1[df1.nettPrice.notnull()])

#df1 = df1.drop(['nettPrice','x','y'],axis=1)

print(df1.info())
print(df1.head())

,area,floorRange,noOfUnits,contractDate,typeOfSale,price,propertyType,district,typeOfArea,tenure,nettPrice,street,x,y,project,Unit Price psf
324,473,51-55,1,2018-09,1,1464000,Apartment,01,Strata,99 yrs lease commencing from 2011,1454000,SHENTON WAY,29765.3992,28838.9204,V ON SHENTON,3095
343,505,51-55,1,2018-08,1,1616000,Apartment,01,Strata,99 yrs lease commencing from 2011,1567520,SHENTON WAY,29765.3992,28838.9204,V ON SHENTON,3200
349,1205,41-45,1,2018-10,1,2654505,Apartment,01,Strata,99 yrs lease commencing from 2011,2644505,SHENTON WAY,29765.3992,28838.9204,V ON SHENTON,2202
351,1205,41-45,1,2019-02,1,2855680,Apartment,01,Strata,99 yrs lease commencing from 2011,2847680,SHENTON WAY,29765.3992,28838.9204,V ON SHENTON,2369
363,1205,41-45,1,2018-10,1,2677185,Apartment,01,Strata,99 yrs lease commencing from 2011,2667185,SHENTON WAY,29765.3992,28838.9204,V ON SHENTON,2221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64769,1065,01-05,1,2018-04,1,914000,Executive Condominium,27,Strata,99 yrs lease commencing from 2014,911700,SEMBAWANG CRESCENT,26404.5082,47225.8473,PARC LIFE,858
64780,1000,01-05,1,2018-04,1,846000,Executive Condominium,27,Strata,99 yrs lease commencing from 2014,843700,SEMBAWANG CRESCENT,26404.5082,47225.8473,PARC LIFE,846
64782,1065,01-05,1,2018-04,1,913000,Executive Condominium,27,Strata,99 yrs lease commencing from 2014,910700,SEMBAWANG CRESCENT,26404.5082,47225.8473,PARC LIFE,857
64783,763,01-05,1,2018-10,1,828000,Executive Condominium,27,Strata,99 yrs lease commencing from 2014,825700,SEMBAWANG CRESCENT,26404.5082,47225.8473,PARC LIFE,1085


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65022 entries, 0 to 65021
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype    
---  ------          --------------  -----    
 0   area            65022 non-null  int32    
 1   floorRange      65022 non-null  category 
 2   noOfUnits       65022 non-null  int32    
 3   contractDate    65022 non-null  period[M]
 4   typeOfSale      65022 non-null  category 
 5   price           65022 non-null  int32    
 6   propertyType    65022 non-null  category 
 7   district        65022 non-null  category 
 8   typeOfArea      65022 non-null  category 
 9   tenure          65022 non-null  object   
 10  nettPrice       245 non-null    object   
 11  street          65022 non-null  object   
 12  x               43740 non-null  float64  
 13  y               43740 non-null  float64  
 14  project         65022 non-null  object   
 15  Unit Price psf  65022 non-null  int32    
dtypes: category(5), float64(2), int32(4), ob

In [8]:
#display(df[df['project'].str.contains('CARIBBEAN')].head(20))

In [9]:

print(Counter(df1.floorRange))
print(Counter(df1.typeOfSale))
print(Counter(df1.propertyType))
print(Counter(df1.district))
print(Counter(df1.typeOfArea))

Counter({'01-05': 21660, '06-10': 15321, '11-15': 10580, '-': 6763, '16-20': 5180, '21-25': 2313, '26-30': 1377, '31-35': 956, '36-40': 527, '41-45': 187, '46-50': 58, '51-55': 35, '56-60': 23, '61-65': 18, '66-70': 14, 'B1-B5': 7, '71-75': 3})
Counter({'3': 35229, '1': 28987, '2': 806})
Counter({'Condominium': 27602, 'Apartment': 25739, 'Executive Condominium': 4965, 'Terrace': 3279, 'Semi-detached': 1723, 'Detached': 771, 'Strata Terrace': 720, 'Strata Semi-detached': 178, 'Strata Detached': 45})
Counter({'19': 10358, '18': 4582, '15': 4446, '14': 4316, '05': 4272, '03': 3942, '10': 3454, '23': 2791, '09': 2742, '13': 2660, '21': 2462, '20': 2323, '27': 2291, '16': 2103, '28': 2013, '11': 1803, '12': 1573, '17': 1237, '22': 886, '07': 788, '01': 746, '04': 741, '25': 687, '02': 640, '08': 602, '26': 560, '06': 4})
Counter({'Strata': 59237, 'Land': 5785})


In [10]:
print(Counter(df1.tenure))

this_year = dt.date.today().year

for i in range(df.shape[0]):
    if (df.loc[i,'tenure'] == 'Freehold') or (df.loc[i,'tenure'][0:3] == '9999'):
        df.loc[i,'lease left'] = 9999
    else:
        lease = int(df.loc[i,'tenure'][0:2])
        df.loc[i,'lease left'] = lease - this_year + df.loc[i,'tenure'][-4:] 

Counter({'Freehold': 19164, '99 yrs lease commencing from 2018': 11464, '99 yrs lease commencing from 2017': 4657, '99 yrs lease commencing from 2016': 2919, '99 yrs lease commencing from 2011': 2826, '99 yrs lease commencing from 2010': 2018, '99 yrs lease commencing from 2019': 1982, '99 yrs lease commencing from 2014': 1647, '99 yrs lease commencing from 2012': 1584, '99 yrs lease commencing from 2015': 1495, '99 yrs lease commencing from 1997': 1421, '99 yrs lease commencing from 2008': 1043, '99 yrs lease commencing from 1996': 898, '99 yrs lease commencing from 1995': 772, '99 yrs lease commencing from 2007': 729, '99 yrs lease commencing from 2000': 727, '99 yrs lease commencing from 2013': 718, '99 yrs lease commencing from 2006': 624, '99 yrs lease commencing from 1994': 541, '99 yrs lease commencing from 2002': 471, '99 yrs lease commencing from 2001': 470, '99 yrs lease commencing from 1993': 448, '99 years leasehold': 419, '99 yrs lease commencing from 2009': 399, '99 yrs l

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [ ]:
df1.to_csv("d:/downloads/ura_caveats_2017-10 to 2020-09")

In [ ]:
df = pd.read_csv("d:/downloads/ura_caveats_2017-10 to 2020-09", parse_dates=['contractDate'])

In [ ]:
df.info()

In [ ]:
df.loc[df.nettPrice.notnull(),'price'] = df.loc[df.nettPrice.notnull(),'nettPrice']
display(df[df.nettPrice.notnull()].sample(10))

In [ ]:
display(df.sample(10))